In [ ]:
!pip install SCALECAST

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from scipy.stats import norm
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from scipy.optimize import curve_fit
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import matplotlib.pyplot as plt;
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_percentage_error as mape
from scalecast.Forecaster import Forecaster
plt.rcdefaults()

## LSTM

In [ ]:

df_uah1 = pd.read_excel("/content/drive/MyDrive/diploma/Data_1_S.xlsx", "quantity")
df_uah1["Total 1"] = df_uah1["T1"] +df_uah1["T2"] +df_uah1["T3"] +df_uah1["T4"]
df_uah1["Total 2"] = df_uah1[[c for c in df_uah1.columns if "P" in c]].sum(axis=1)
df_uah1["Total"] = df_uah1["Total 1"] + df_uah1["Total 2"]
test_size = 0.1
full_test_size = 131 #131
train = df_uah1.iloc[:int(len(df_uah1)*(1-test_size))]
test = df_uah1.iloc[int(len(df_uah1)*(1-test_size)):full_test_size]

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:211: UserWarning: Cell A2 is marked as a date but the serial value 227278601.6 is outside the limits for dates. The cell will be treated as an error.
  warn(msg)
/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:211: UserWarning: Cell A3 is marked as a date but the serial value 99929344.48 is outside the limits for dates. The cell will be treated as an error.
  warn(msg)
<ipython-input-4-fca1782498b0>:5: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_uah1["Total 2"] = df_uah1[[c for c in df_uah1.columns if "P" in c]].sum(axis=1)


In [ ]:
f = Forecaster(
    y=df_uah1["Total"],
    current_dates=df_uah1['Period'],
    test_length = int(len(df_uah1)*test_size),
    future_dates = 24,
    cis = False,
)


In [ ]:
f.set_estimator('lstm')
f.manual_forecast(
    call_me='lstm_dropout0.3_Adam_l5=0.001',
    lags=12,
    batch_size=16,
    epochs=300,
    validation_split=.2,
    shuffle=False,
    activation='tanh',
    optimizer='Adam',
    learning_rate=0.001,
    lstm_layer_sizes=(256,)*15,
    dropout=(0,)*15,
    plot_loss=True,
)

# f.plot_test_set()
# plt.title('Default Test-set Performance',size=16)
# plt.show()

In [ ]:
f.export('model_summaries',determine_best_by='TestSetMAPE')[
    ['ModelNickname','TestSetMAPE','TestSetRMSE','TestSetR2','best_model']
]



,ModelNickname,TestSetMAPE,TestSetRMSE,TestSetR2,best_model
0,lstm_dropout0.3_Adam_l5=0.001,0.270928,6.260673e+07,-0.463818,True


## Mulivariate regression with Scalecast

In [ ]:
exog = pd.read_csv("/content/drive/MyDrive/diploma/x_projections.csv").set_index("Unnamed: 0")
# exog[["CPI", "GDP", "Population", "Internet_access"]].isna()
exog
cols = ["Average_price_CPI_adjusted_projected", "Internet_access_smoothed_predicted", "Population_smoothed_predicted", "GDP_smoothed_predicted", "CPI_smoothed_predicted", "customer_pain1", "customer_pain2","customer_pain3","customer_pain4","customer_pain5","customer_pain6",]

In [ ]:
f2 = Forecaster(
    y=df_uah1["Total"],
    current_dates=df_uah1['Period'],
    test_length = len(df_uah1)*test_size,
    future_dates = 24,
    cis = False,
)


from scalecast.SeriesTransformer import SeriesTransformer

transformer = SeriesTransformer(f2,deepcopy=False)
f2 = transformer.DiffTransform()

f2.add_ar_terms(24)
# f2.add_seasonal_regressors('month',dummy=True)
for c in cols:
  f2.add_series(exog[c].iloc[:len(df_uah1)], called=c)
# f2.add_seasonal_regressors('year')
# f2.add_time_trend()

f2.set_estimator('mlr')
f2.manual_forecast()

In [ ]:
f2 = transformer.DiffRevert(
    exclude_models = [m for m in f2.history if m != 'mlr']
) # exclude all lstm models already called from the revert

In [ ]:
# f2.plot_test_set(order_by='TestSetMAPE',models=['mlr'])
# plt.title('Top-2 Models Test-set Performance - Level Data',size=16)
# plt.show()

In [ ]:
f2.export('model_summaries',determine_best_by='TestSetMAPE')[
    ['ModelNickname','TestSetMAPE','TestSetRMSE','TestSetR2','best_model']
]

,ModelNickname,TestSetMAPE,TestSetRMSE,TestSetR2,best_model
0,mlr,0.37933,1.111312e+08,-18.324681,True


In [ ]:
# mlr  - 0.37933 ;85 - 0.5442	 ; 90 - 0.209124
!pip install shap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.6/572.6 kB 9.4 MB/s eta 0:00:00


In [ ]:
import shap
f2.save_feature_importance()
f2.export_feature_importance('mlr')
# a

,weight,std
feature,,
AR5,0.821532,0.139477
AR4,0.775098,0.148809
Average_price_CPI_adjusted_projected,0.755008,0.139701
AR7,0.631565,0.030241
AR6,0.581671,0.154332
Population_smoothed_predicted,0.507927,0.299289
AR11,0.440200,0.090793
Internet_access_smoothed_predicted,0.367457,0.144609
AR3,0.325255,0.128763
